# Reading Excel Files and Wrapping it into a Report

Assumptions:

* Measurements will be removed for ease of cleaning.
* JC is the parent
* SF-206 is the child
* The structure is fixed space. Some values (magic numbers) are hardcoded for ease of programming.


If more than one match ever pops up when looking at abundance, it will pick the first one.
If no match comes up, it will return empty string

In [1]:
import pandas as pd
import os
import glob
from datetime import datetime


In [2]:
import re


## Helper Functions and Classes

The following are functions and classes to help have more readable code

In [3]:
### class to hold the Spectra and Test data

class massSpecData:
    def __init__(self, 
                 data_file,
                 sample_name,
                 user_name,
                 acquired_time,
                 acq_method,
                 da_method,
                 source_biomolecule_spectrum_peak_list):

        self.data_file = data_file
        self.sample_name = sample_name
        self.user_name = user_name
        self.acquired_time = acquired_time
        self.acq_method = acq_method
        self.da_method = da_method
        self.source_biomolecule_spectrum_peak_list = source_biomolecule_spectrum_peak_list
        
    
    def __str__(self):
        s = "Header File:\n----" + "\n" + \
        "Data File: " + self.data_file + "\n" + \
        "Sample Name: " + self.sample_name + "\n" + \
        "User Name: " + self.user_name + "\n" + \
        "Acquired Time: " + self.acquired_time + "\n" + \
        "Acq Method: " + self.acq_method + "\n" + \
        "DA Method: " + self.da_method + "\n" 
        return s

In [4]:
# add future measurements and text removal here

def filterMeasurements(s, textRemoval):
    #textRemoval = ['3uM', '300uM']
    for text in textRemoval:
        s = re.sub(text,'',s)
    return s
    

In [5]:
# this currently only selects the first match, which hopefully is good enough

def findAbundance(df, x, tolerance):
    searchDF = df[(df['Neutral Mass'] >= x-tolerance)&(df['Neutral Mass'] <= x+tolerance)]
    if searchDF.shape[0] > 0:
        searchDF = searchDF.reset_index(drop=True)
        return(searchDF.at[0,'Abundance'])
    else:
        return ""
        
    

In [6]:
def generateLine(data_file, sample_name, x_array, tolerance, relational_values, text_removal, numBound, source_biomolecule_spectrum_peak_list):
    
    yMultiple = []
    
    
    data = {'Data File': data_file}
    data['Sample Name']= sample_name
    
    xBound = []
    xyBound = []
    
    xCounter = 0
    for x in x_array:
        abundance = findAbundance(source_biomolecule_spectrum_peak_list, x, tolerance)
        data['X'+str(xCounter)+': '+str(x)+"+/-" +str(tolerance)] = [abundance]
        if abundance == '':
            xBound.append(0)
        else:
            xBound.append(abundance)
        xCounter = xCounter+1
    
    name = filterMeasurements(sample_name,text_removal) #remove the measurements
    splitName = name.split("+") # split by +
    
    # # not just the baseline
    if len(splitName) > 1: 
        child_name = splitName[-1]
        child_name = child_name.strip()
        y = findY(child_name, relational_values)
        
        data['Child Name'] = child_name
        data['Y']= y
        xCounter = 0
        
        for x in x_array:
            yCounter = 0
            for multiplicationFactor in numBound:
                abundance = findAbundance(source_biomolecule_spectrum_peak_list, x + multiplicationFactor*y, tolerance)
                data ['(X'+str(xCounter)+'+'+str(multiplicationFactor)+'Y): ('+str(x)+"+/-" +str(tolerance)+"+"+str(multiplicationFactor)+"*Y)"] =  [abundance]
                
                #xyBound.append(float(abundance))
                
                if abundance == '':
                    xyBound.append(0)
                else:
                    xyBound.append(abundance)
                yCounter += 1
                
            xCounter = xCounter + 1
        
        nominator = 0
        denominator = 0
        try:
            for value in xBound:
                denominator += value

            for value in xyBound:
                nominator += value
                denominator += value
        
            data['% Bound'] = nominator/denominator*100
        except:
            #print("there was some error")
            data['% Bound'] = ''

    # this is if it is just the baseline, no child
    else:
        data['Child Name'] = ''
        data['Y'] = ''
        
        xCounter = 0
        for x in x_array:
            yCounter = 0
            for multiplicationFactor in numBound:
                #data ['(X'+str(xCounter)+'+Y): ('+str(x)+"+/-" +str(tolerance)+"+"+str(multiplicationFactor)+"*Y)"] =  ['']
                data ['(X'+str(xCounter)+'+'+str(multiplicationFactor)+'Y): ('+str(x)+"+/-" +str(tolerance)+"+"+str(multiplicationFactor)+"*Y)"] =  ['']
                yCounter += 1
                
            xCounter = xCounter + 1
        data['% Bound'] = ''
    
    df = pd.DataFrame(data)
    return df

In [7]:
## use this if you don't want to use a lookup table located in "relationalValues.csv"

# def findY(childName):
    
    
#     childLookupTable = {
#         'SF-206': 340 # add more here, or use some pandas dataframe to do the key in and value out
        
        
#     }
    
#     try:
#         Y = childLookupTable[childName]
#     except:
#         Y = 0
#         print ("child not found in lookup table, please add to the table")
#         print ("A value of 0 is used in the meanwhile")
        
    
#     return Y

In [8]:
def findY(childName, df):

    #df = pd.read_csv("relationalValues.csv")
    
    
    childLookupTable = {}
    
    for i in range(df.shape[0]):
        childLookupTable[df['child'].values[i]]=df['weight'].values[i]
    
    try:
        Y = childLookupTable[childName]
    except:
        Y = 0
        print ("child not found in lookup table, please add to the table")
        print ("A value of 0 is used in the meanwhile")
        
    
    return Y
    
    

## Edit codeblock here

This box should be all that is needed to be edited. Everything else should be fine. One thing that is of issue is the tolerance. If nothing is found, then the bound calculations will not work.

In [9]:
[i for i in range(1,6,1)]

[1, 2, 3, 4, 5]

In [10]:
# change relational filepath here
relational_values = pd.read_csv("relationalValues.csv")

path = "C:/Users/Marvin L/Downloads/mass_spec_analysis/"
parent = 'JC'
# child = ['SF-206']

tolerance = 5
x_array = [31169, 31212]
#x_array = [19165]
#x_array = [31169]
#x_array = [31169, 31212, 31512, 31555 ]

text_removal = ['3uM', '300uM'] # add any text you want removed here

# removal of '+' will be implicit

minNumBound = 1
maxNumBound = 6
boundIncrement = 1

numBound = [i for i in range(minNumBound, maxNumBound+1, boundIncrement)]

# can also comment this out if you want to note a specific numer sequence
# numBound = [1, 2, 3, 4, 5, 6]


# autogenerate datetime
now = datetime.now()
date_time = now.strftime("%Y%m%d_%H%M%S")


filename_output = "analysis_" + date_time
filename_output += "_" + parent
for x in x_array:
    filename_output += "_X" + str(x)
filename_output += "_+-" + str(tolerance)
filename_output += ".xlsx" #filename_output is analysis_YYYYMMDD_HHMMSS_X0_X1

#filename_output = "analysisOutput2.xlsx" # can reenable this to overwrite autogenerated name

## Begin Script

In [11]:
extension = 'xls'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

In [12]:
massSpecData_list = [] # make a list of massSpecData

In [13]:
# populate the massSpecData_list
for file in result:
    
    filepath = path+file
    df = pd.read_excel(filepath)
    df.columns = ['c1','c2','c3','c4','c5']
    
    data_file = df.at[0,'c2']
    sample_name = df.at[0, 'c4']
    user_name = df.at[2, 'c4']
    acquired_time = df.at[3, 'c4']
    acq_method = df.at[3,'c2']
    da_method = df.at[4, 'c4']
    # create the dataframe
    source_biomolecule_spectrum_peak_list = df[13:33]
    source_biomolecule_spectrum_peak_list = source_biomolecule_spectrum_peak_list.drop(['c3','c4','c5'], axis=1)
    source_biomolecule_spectrum_peak_list = source_biomolecule_spectrum_peak_list.reset_index(drop=True)
    source_biomolecule_spectrum_peak_list.columns = ['Neutral Mass','Abundance']

    tempMassSpecData = massSpecData(
                 data_file,
                 sample_name,
                 user_name,
                 acquired_time,
                 acq_method,
                 da_method,
                 source_biomolecule_spectrum_peak_list)

    massSpecData_list.append(tempMassSpecData)

In [14]:
# make and process each of the files
df_list = []
for ms in massSpecData_list:
    
    data_file = ms.data_file
    sample_name = ms.sample_name
    source_biomolecule_spectrum_peak_list = ms.source_biomolecule_spectrum_peak_list
    
    
    temp_df = generateLine(data_file, sample_name, x_array, tolerance, relational_values, text_removal, numBound, source_biomolecule_spectrum_peak_list)
    df_list.append(temp_df)

In [15]:
mergedDF = pd.concat(df_list)

In [16]:
mergedDF = mergedDF.reset_index(drop=True)

In [17]:
mergedDF

,Data File,Sample Name,X0: 31169+/-5,X1: 31212+/-5,Child Name,Y,(X0+1Y): (31169+/-5+1*Y),(X0+2Y): (31169+/-5+2*Y),(X0+3Y): (31169+/-5+3*Y),(X0+4Y): (31169+/-5+4*Y),(X0+5Y): (31169+/-5+5*Y),(X0+6Y): (31169+/-5+6*Y),(X1+1Y): (31212+/-5+1*Y),(X1+2Y): (31212+/-5+2*Y),(X1+3Y): (31212+/-5+3*Y),(X1+4Y): (31212+/-5+4*Y),(X1+5Y): (31212+/-5+5*Y),(X1+6Y): (31212+/-5+6*Y),% Bound
0,Inj00055-1-A23-JC 3uM.d,JC 3uM,32862.05,35635.72,,,,,,,,,,,,,,,
1,Inj00057-1-A24-JC 3uM.d,JC 3uM,37142.39,39068.42,,,,,,,,,,,,,,,
2,Inj00059-1-A25-JC 3uM.d,JC 3uM,37065.01,42276.44,,,,,,,,,,,,,,,
3,Inj00061-1-A26-JC 3uM.d,JC 3uM,40395.66,42496.77,,,,,,,,,,,,,,,
4,Inj00063-1-A27-JC 3uM + SF-206 300uM.d,JC 3uM + SF-206 300uM,31792.53,33248.90,SF-206,343,7531.06,,,,,,9041.28,,,,,,20.305814
5,Inj00065-1-A28-JC 3uM + SF-206 300uM.d,JC 3uM + SF-206 300uM,31389.92,33965.52,SF-206,343,8632.73,,,,,,,,,,,,11.667717
6,Inj00067-1-A29-JC 3uM + SF-206 300uM.d,JC 3uM + SF-206 300uM,31179.26,33273.60,SF-206,343,7487.97,,,,,,7905.41,,,,,,19.278779
7,Inj00069-1-A30-JC 3uM + SF-206 300uM.d,JC 3uM + SF-206 300uM,33759.98,33271.19,SF-206,343,8415.68,,,,,,7818.52,,,,,,19.496941


In [18]:
mergedDF.to_excel(filename_output,index=False)

In [19]:
filename_output

'analysis_20230614_235950_JC_X31169_X31212_+-5.xlsx'